In [1]:
import os
import csv
from PIL import Image
import torch.nn as nn
import torch
from torchvision import transforms
import torch.nn.functional as F

In [2]:
# --- Attention Block ---
class SEBlock(nn.Module):
    def __init__(self, channels, reduction=16):
        super(SEBlock, self).__init__()
        self.fc1 = nn.Linear(channels, channels // reduction)
        self.fc2 = nn.Linear(channels // reduction, channels)

    def forward(self, x):
        b, c, _, _ = x.size()
        y = F.adaptive_avg_pool2d(x, 1).view(b, c)
        y = F.relu(self.fc1(y))
        y = torch.sigmoid(self.fc2(y)).view(b, c, 1, 1)
        return x * y

# --- Conv Block with Dropout + Attention ---
def conv_block(in_c, out_c, dropout=0.0):
    layers = [
        nn.Conv2d(in_c, out_c, 3, padding=1),
        nn.BatchNorm2d(out_c),
        nn.ReLU(inplace=True),
        nn.Conv2d(out_c, out_c, 3, padding=1),
        nn.BatchNorm2d(out_c),
        nn.ReLU(inplace=True),
    ]
    if dropout > 0:
        layers.append(nn.Dropout(dropout))
    layers.append(SEBlock(out_c))
    return nn.Sequential(*layers)

# --- Full UNet Model ---
class UNet(nn.Module):
    def __init__(self, in_channels=3, out_channels=233):
        super(UNet, self).__init__()

        self.enc1 = conv_block(in_channels, 64)
        self.enc2 = conv_block(64, 128)
        self.enc3 = conv_block(128, 256)
        self.enc4 = conv_block(256, 512)

        self.pool = nn.MaxPool2d(2)

        self.bottleneck = conv_block(512, 1024)

        self.up3 = nn.ConvTranspose2d(1024, 512, kernel_size=2, stride=2)
        self.dec3 = conv_block(1024, 512, dropout=0.3)

        self.up2 = nn.ConvTranspose2d(512, 256, kernel_size=2, stride=2)
        self.dec2 = conv_block(512, 256, dropout=0.2)

        self.up1 = nn.ConvTranspose2d(256, 128, kernel_size=2, stride=2)
        self.dec1 = conv_block(256, 128, dropout=0.1)

        self.up0 = nn.ConvTranspose2d(128, 64, kernel_size=2, stride=2)
        self.dec0 = conv_block(128, 64)

        self.final = nn.Conv2d(64, out_channels, kernel_size=1)

    def forward(self, x):
        e1 = self.enc1(x)
        e2 = self.enc2(self.pool(e1))
        e3 = self.enc3(self.pool(e2))
        e4 = self.enc4(self.pool(e3))

        b = self.bottleneck(self.pool(e4))

        d3 = self.up3(b)
        d3 = torch.cat([d3, e4], dim=1)
        d3 = self.dec3(d3)

        d2 = self.up2(d3)
        d2 = torch.cat([d2, e3], dim=1)
        d2 = self.dec2(d2)

        d1 = self.up1(d2)
        d1 = torch.cat([d1, e2], dim=1)
        d1 = self.dec1(d1)

        d0 = self.up0(d1)
        d0 = torch.cat([d0, e1], dim=1)
        d0 = self.dec0(d0)

        out = self.final(d0)
        return F.softmax(out, dim=1)


In [3]:
import torch
import torch.nn as nn
import torch.nn.functional as F

class BasicBlock(nn.Module):
    expansion = 1

    def __init__(self, in_channels, out_channels, stride=1, downsample=None):
        super().__init__()
        self.conv1 = nn.Conv2d(in_channels, out_channels, kernel_size=3, stride=stride,
                               padding=1, bias=False)
        self.bn1 = nn.BatchNorm2d(out_channels)

        self.conv2 = nn.Conv2d(out_channels, out_channels, kernel_size=3, stride=1,
                               padding=1, bias=False)
        self.bn2 = nn.BatchNorm2d(out_channels)

        self.downsample = downsample

    def forward(self, x):
        identity = x

        out = F.relu(self.bn1(self.conv1(x)))
        out = self.bn2(self.conv2(out))

        if self.downsample is not None:
            identity = self.downsample(x)

        out += identity
        return F.relu(out)

class ResNet18(nn.Module):
    def __init__(self, num_classes=50):
        super().__init__()
        self.in_channels = 64

        self.conv1 = nn.Conv2d(3, 64, kernel_size=7, stride=2, padding=3, bias=False)
        self.bn1 = nn.BatchNorm2d(64)

        self.maxpool = nn.MaxPool2d(kernel_size=3, stride=2, padding=1)

        self.layer1 = self._make_layer(64, 2)
        self.layer2 = self._make_layer(128, 2, stride=2)
        self.layer3 = self._make_layer(256, 2, stride=2)
        self.layer4 = self._make_layer(512, 2, stride=2)

        self.avgpool = nn.AdaptiveAvgPool2d((1, 1))
        self.fc = nn.Linear(512, num_classes)

    def _make_layer(self, out_channels, blocks, stride=1):
        downsample = None

        if stride != 1 or self.in_channels != out_channels:
            downsample = nn.Sequential(
                nn.Conv2d(self.in_channels, out_channels,
                          kernel_size=1, stride=stride, bias=False),
                nn.BatchNorm2d(out_channels)
            )

        layers = [BasicBlock(self.in_channels, out_channels, stride, downsample)]
        self.in_channels = out_channels

        for _ in range(1, blocks):
            layers.append(BasicBlock(out_channels, out_channels))

        return nn.Sequential(*layers)

    def forward(self, x):
        x = F.relu(self.bn1(self.conv1(x)))
        x = self.maxpool(x)

        x = self.layer1(x)
        x = self.layer2(x)
        x = self.layer3(x)
        x = self.layer4(x)

        x = self.avgpool(x)
        x = torch.flatten(x, 1)
        return self.fc(x)


In [ ]:
import os
import csv
from PIL import Image
import torch
from torchvision import transforms

def generate_submission(classification_model_path, segmentation_model_path, test_dir="test", output_csv="submission.csv", use_dummy_rle=False):
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

    # Load classification model
    classification_model = torch.load(classification_model_path, map_location=device, weights_only=False)
    classification_model.to(device)
    classification_model.eval()

    if not use_dummy_rle:
        # Load segmentation model only if needed
        segmentation_model = torch.load(segmentation_model_path, map_location=device, weights_only=False)
        segmentation_model.to(device)
        segmentation_model.eval()

    transform = transforms.Compose([
        transforms.Resize((224, 224)),
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.485, 0.456, 0.406],
                             std=[0.229, 0.224, 0.225]),
    ])

    def mask_to_rle(mask):
        pixels = mask.flatten()
        if len(pixels) == 0:
            return ""
        counts = []
        prev = pixels[0]
        count = 1
        for pix in pixels[1:]:
            if pix == prev:
                count += 1
            else:
                counts.append(str(count))
                counts.append(str(prev))
                count = 1
                prev = pix
        counts.append(str(count))
        counts.append(str(prev))
        return " ".join(counts)

    rows = [("ID", "Label", "MASK_RLE")]
    test_files = sorted([f for f in os.listdir(test_dir) if f.endswith(".JPEG")])

    for fname in test_files:
        img_path = os.path.join(test_dir, fname)
        image = Image.open(img_path).convert("RGB")
        input_tensor = transform(image).unsqueeze(0).to(device)

        with torch.no_grad():
            class_logits = classification_model(input_tensor)
            pred_class = torch.argmax(class_logits, dim=1).item()

        if use_dummy_rle:
            rle = "50176 0"
        else:
            with torch.no_grad():
                seg_logits = segmentation_model(input_tensor)
                seg_pred = torch.argmax(seg_logits, dim=1).squeeze(0).cpu().numpy()
            rle = mask_to_rle(seg_pred)

        rows.append((fname, pred_class, rle))

    with open(output_csv, "w", newline="", encoding="utf-8") as f:
        writer = csv.writer(f)
        writer.writerows(rows)

    print(f"Submission CSV created at {output_csv} ({'dummy RLE' if use_dummy_rle else 'full RLE'})")

# Example usage:
generate_submission("saved_models/resnet18v6.pth", "saved_models/unetv2.pth", use_dummy_rle=False)


Submission CSV created at submission.csv (dummy RLE)
